In [1]:
# JUST RUN ALL CELLS AND CHECK THE RESULTS AT THE BOTTOM

import argparse
import csv
import os
import shutil
from tqdm import tqdm

try: import setGPU
except ImportError: pass

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import numpy as np
import numpy.random as npr

import sys
import time

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('bmh')

import models

import sys

import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

batchSz = 64

In [2]:
model = torch.load("work/mnist.lenet-optnet.nHidden:50.nineq:100.eps:0.0001/latest.pth", map_location=torch.device('cpu'))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'models.LenetOptNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [3]:
testLoader = torch.utils.data.DataLoader(
    dset.MNIST('data/mnist', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batchSz, shuffle=False, **{})

In [4]:
def test(net):
    net.eval()
    test_loss = 0
    incorrect = 0

    start_time = time.time()
    with torch.no_grad():
        i = 0
        for data, target in testLoader:
            i += 1
            print('Testing model: {}/{}'.format(i, len(testLoader)), end='\r')
            data, target = Variable(data), Variable(target)
            output = net(data)
            test_loss += F.nll_loss(output, target).data
            pred = output.data.max(1)[1] # get the index of the max log-probability
            incorrect += pred.ne(target.data).cpu().sum()
    
    dt = time.time() - start_time
    test_loss = test_loss
    test_loss /= len(testLoader) # loss function already averages over batch size
    nTotal = len(testLoader) * batchSz
    err = 100.*incorrect/nTotal
    print('\nTest set: Average loss: {:.4f}, Error: {}/{} ({:.0f}%)\n'.format(
        test_loss, incorrect, nTotal, err))
    print(f'Average time per batch: {dt / len(testLoader)}')

In [5]:
model1 = models.LenetDynamic(nineq=100)

params = model.named_parameters()
params1 = model1.named_parameters()

dict_params1 = dict(params1)

for name, param in params:
    if name in dict_params1:
        dict_params1[name].data.copy_(param.data)

AssertionError: Torch not compiled with CUDA enabled

In [6]:
# OPTNET PERFORMANCE TEST
test(model)

AssertionError: Torch not compiled with CUDA enabled

In [7]:
# DYNAMIC PERFORMANCE TEST
test(model1)

NameError: name 'model1' is not defined